In [1]:
from pynq import Overlay
from pynq import allocate
import numpy as np

In [2]:
ol = Overlay("stream_fp_int.bit")

In [3]:
ol?


In [4]:
img=np.load("img.npy")
label=np.load("label.npy")

In [5]:
inBuffer0 = allocate(shape=(784,), dtype=np.float32)
outBuffer= allocate(shape=(10,), dtype=np.float32)
dma=ol.axi_dma_0


In [8]:
dma.sendchannel.transfer(inBuffer0)
dma.recvchannel.transfer(outBuffer)
ol.top_0.write(0x00, 0x01)
while(ol.top_0.register_map.CTRL.AP_DONE==0):
    pass
print(outBuffer)

[ -20.174593    -8.319537    18.026855    -0.5553475 -102.96447
   17.360079   -41.692043     1.7244928  -38.64471    -56.771233 ]


In [126]:
print(outBuffer.argmax())
print(label[2])

4
1.0


# Load Label, Img

In [9]:
label=np.load("label.npy")
img=np.load("img.npy")

In [10]:
import time

# Test accuracy

In [11]:
hit=0
start_time=time.time()
for i in range(1000):
    inBuffer0[:]=img[i]
    dma.sendchannel.transfer(inBuffer0)
    dma.recvchannel.transfer(outBuffer)
    ol.top_0.write(0x00, 0x01)
    while(ol.top_0.register_map.CTRL.AP_DONE==0):
        pass
    if(outBuffer.argmax()==label[i]):
        hit+=1
    #print("times: ",i+1,"hit: ",hit)
print("acc",hit/1000)
print("spend time",time.time()-start_time)

acc 0.974
spend time 20.93618106842041
